In [ ]:
import numpy as np
import pprint as pp
from matplotlib import pyplot as plt
import shutil
import os
import copy
from typing import List, Tuple, Dict, Any
from IPython.display import clear_output
import time
from tqdm.notebook import tqdm
import random
import pandas as pd

VISUALIZE=False

### Utils

In [ ]:
def read_data_file(file_name: str) -> Dict[Any, Any]:
    tsp_data = {}
    with open(FILENAME, 'r') as file:
        for line in file:
            if line.startswith('EOF'):
                break
            if line.startswith('DIMENSION'):
                tsp_data['DIMENSION'] = int(line.split()[1])
            parts = line.split()
            if parts[0].isdigit():
                node, x, y = map(int, parts)
                tsp_data[node] = (x, y)

    return tsp_data

def calculate_distance_matrix(tsp_data: Dict[Any, Any]) -> np.ndarray:
    N = tsp_data['DIMENSION']
    tsp_data.pop('DIMENSION')

    distance_matrix = [[0 for _ in range(N + 1)] for _ in range(N + 1)]
    for node in tsp_data:
        x1, y1 = tsp_data[node]
        for other_node in tsp_data:
            if node == other_node:
                continue
            else:
                x2, y2 = tsp_data[other_node]
                distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                distance_matrix[node][other_node] = distance
                distance_matrix[other_node][node] = distance
    return np.array(distance_matrix)

def print_distance_matrix(distance_matrix: np.ndarray):
    print(distance_matrix)

In [ ]:
def plot_cities(data: Dict[Any, Any]) -> None:
    x = []
    y = []
    for city in data:
        x.append(data[city][0])
        y.append(data[city][1])
    plt.scatter(x, y)
    plt.show()


def plot_cycles(cycle1: List, cycle2: List, tsp_data: Dict[Any, Any], file_name) -> None:
    plt.figure(figsize=(8, 6))

    for city in cycle1:
        plt.scatter(tsp_data[city][0], tsp_data[city][1], color='red', s=30)
        plt.text(tsp_data[city][0], tsp_data[city][1], str(city), fontsize=8, ha='left', va='bottom')

    cycle1_x = [tsp_data[city][0] for city in cycle1]
    cycle1_y = [tsp_data[city][1] for city in cycle1]
    cycle1_x.append(cycle1_x[0])
    cycle1_y.append(cycle1_y[0])
    plt.plot(cycle1_x, cycle1_y, linestyle='-', color='blue', label='Cycle 1')

    for city in cycle2:
        plt.scatter(tsp_data[city][0], tsp_data[city][1], color='red', s=30)
        plt.text(tsp_data[city][0], tsp_data[city][1], str(city), fontsize=8, ha='left', va='bottom')

    cycle2_x = [tsp_data[city][0] for city in cycle2]
    cycle2_y = [tsp_data[city][1] for city in cycle2]
    cycle2_x.append(cycle2_x[0])
    cycle2_y.append(cycle2_y[0])
    plt.plot(cycle2_x, cycle2_y, linestyle='-', color='green', label='Cycle 2')

    plt.title('Visualization of Cycles')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.grid(True)
    plt.legend()
    plt.savefig(file_name)
    plt.show()


In [ ]:
def get_nodes(tsp_data: Dict[Any, Any]) -> np.ndarray:
    nodes = []
    for node in tsp_data:
        if node != 'DIMENSION':
            nodes.append(node)
    return np.array(nodes)

In [ ]:
def swap_nodes_between_cycles(cycle1: List, cycle2: List, a_index: int, b_index: int) -> Tuple[np.ndarray, np.ndarray]:
    # cycle1 = copy.deepcopy(cycle1)
    # cycle2 = copy.deepcopy(cycle2)

    tmp = cycle2[b_index]
    cycle2[b_index] = cycle1[a_index]
    cycle1[a_index] = tmp

    return cycle1, cycle2


def swap_edge_within_cycle(cycle: List, a_index: int, b_index: int) -> List:
    # cycle = copy.deepcopy(cycle)
    
    b_index = (b_index + 1) % (len(cycle) + 1)
    if a_index > b_index:
        a_index, b_index = b_index, a_index
    cycle[a_index:b_index] = cycle[a_index:b_index][::-1]
    return cycle

In [ ]:
SWAP_NODES_BETWEEN = 2
SWAP_EDGES_WITHIN_CYCLE_1 = 0
SWAP_EDGES_WITHIN_CYCLE_2 = 1

def generate_moves(cycle: List):

    moves = []
    N = len(cycle)

    # generate edges
    for i in range(2, N-1):
        for j in range(N):
            k = (j + i) % (N)
            if j < k:
                moves.append((SWAP_EDGES_WITHIN_CYCLE_1, (j, k)))

    # generate edges
    for i in range(2, N-1):
        for j in range(N):
            k = (j + i) % (N)
            if j < k:
                moves.append((SWAP_EDGES_WITHIN_CYCLE_2, (j, k)))

    # generate nodes
    for i in range(N):
        for j in range(N):
                moves.append((SWAP_NODES_BETWEEN, (i, j)))
    
    # shuffle moves
    np.random.shuffle(moves)
    return moves

In [ ]:
def apply_move(move: Tuple[int, Tuple[int, int]], cycle1: List, cycle2: List) -> Tuple[List, List]:
    move_type, (a, b) = move
    
    if move_type == SWAP_NODES_BETWEEN:
        cycle1, cycle2 = swap_nodes_between_cycles(cycle1, cycle2, a, b)

    elif move_type == SWAP_EDGES_WITHIN_CYCLE_1:
        cycle1 = swap_edge_within_cycle(cycle1, a, b)

    elif move_type == SWAP_EDGES_WITHIN_CYCLE_2:
        cycle2 = swap_edge_within_cycle(cycle2, a, b)

    return cycle1, cycle2

In [ ]:
def calculate_cycles_length(cycle1: List, cycle2: List, distance_matrix: np.ndarray) -> float:
    c = [cycle1, cycle2]
    total_length = 0
    for cycle in c:
        length = 0
        for i in range(len(cycle)):
            length += distance_matrix[cycle[i-1]][cycle[i]]
        total_length += length

    return total_length

### Random TSP Algorithm

In [ ]:
def random_cycle(nodes) -> Tuple[np.ndarray, np.ndarray]:
    nodes = copy.deepcopy(nodes)
    np.random.shuffle(nodes)
    half = len(nodes) // 2
    cycle1 = np.array(nodes[:half])
    cycle2 = np.array(nodes[half:])
    
    return cycle1, cycle2

In [ ]:
def rank_move(move, cycle1, cycle2, distance):
    subdistance_before = 0
    subdistance_after = 0

    move_type, (A, B) = move

    if move_type == SWAP_NODES_BETWEEN:
        a = cycle1[A]
        b = cycle2[B]

        a_prev = cycle1[(A-1) % len(cycle1)]
        a_next = cycle1[(A+1) % len(cycle1)]

        b_prev = cycle2[(B-1) % len(cycle2)]
        b_next = cycle2[(B+1) % len(cycle2)]

        subdistance_before += distance[a_prev][a] + distance[a][a_next] + distance[b_prev][b] + distance[b][b_next]
        subdistance_after += distance[a_prev][b] + distance[b][a_next] + distance[b_prev][a] + distance[a][b_next]

    elif move_type == SWAP_EDGES_WITHIN_CYCLE_1:
        a = cycle1[A]
        b = cycle1[B]

        a_prev = cycle1[(A-1) % len(cycle1)]
        b_next = cycle1[(B+1) % len(cycle1)]

        subdistance_before += distance[a_prev][a] + distance[b][b_next]
        subdistance_after += distance[a_prev][b] + distance[a][b_next]

    elif move_type == SWAP_EDGES_WITHIN_CYCLE_2:

        a = cycle2[A]
        b = cycle2[B]

        a_prev = cycle2[(A-1) % len(cycle2)]
        b_next = cycle2[(B+1) % len(cycle2)]

        subdistance_before += distance[a_prev][a] + distance[b][b_next]
        subdistance_after += distance[a_prev][b] + distance[a][b_next]

    return subdistance_after - subdistance_before

In [ ]:
def local_search_steepest(cycle1, cycle2, distance, data):

    moves = generate_moves(cycle1)

    while True:

        best_move = None
        best_delta = 0

        for move in moves:

            delta = rank_move(move, cycle1, cycle2, distance)
            if delta < best_delta:
                best_move = move
                best_delta = delta
                
        if best_delta < 0:
            cycle1, cycle2 = apply_move(best_move, cycle1, cycle2)
            if VISUALIZE:
                clear_output(wait=True)
                plot_cycles(cycle1, cycle2, data)
        elif best_move is None:
            break

    return cycle1, cycle2


### Hybrid Evolutionary Algorithm

In [ ]:
def hybrid_evolutionary_algorithm(distance_matrix, data, pop_size, num_generations):
    population = []
    
    for _ in range(pop_size):
        individual = random_cycle(get_nodes(data))
        cycle1, cycle2 = local_search_steepest(*individual, distance_matrix, data)
        population.append((cycle1, cycle2))
    
    for _ in range(num_generations):
        parent1, parent2 = np.random.choice(population, 2, replace=False)
        
        offspring = recombine(parent1, parent2)
        
        cycle1, cycle2 = local_search_steepest(*offspring, distance_matrix)
        offspring = (cycle1, cycle2)
        
        worst_individual = max(population, key=lambda ind: calculate_cycles_length(*ind, distance_matrix))
        worst_length = calculate_cycles_length(*worst_individual, distance_matrix)
        offspring_length = calculate_cycles_length(*offspring, distance_matrix)
        
        if offspring_length < worst_length and is_diverse(offspring, population):
            population.remove(worst_individual)
            population.append(offspring)
    
    best_individual = min(population, key=lambda ind: calculate_cycles_length(*ind, distance_matrix))
    return best_individual

def recombine(parent1, parent2): # crossing over
    cycle1 = np.concatenate((parent1[0][:len(parent1[0])//2], parent2[0][len(parent2[0])//2:]))
    cycle2 = np.concatenate((parent1[1][:len(parent1[1])//2], parent2[1][len(parent2[1])//2:]))
    return cycle1, cycle2

def is_diverse(individual, population, threshold=0.1):
    individual_length = calculate_cycles_length(*individual, distance_matrix)
    for other in population:
        other_length = calculate_cycles_length(*other, distance_matrix)
        if abs(individual_length - other_length) < threshold * individual_length:
            return False
    return True

### Run

In [ ]:
FILENAME = 'kroB100.tsp'
VISUALIZE = False
num_of_runs = 3

In [ ]:
data = read_data_file(FILENAME)
distance_matrix = calculate_distance_matrix(data)

In [ ]:
best_cycle_1, best_cycle_2 = hybrid_evolutionary_algorithm(distance_matrix, data, pop_size=50, num_generations=10)
print("Najlepszy cykl 1:", best_cycle_1)
print("Najlepszy cykl 2:", best_cycle_2)